<a href="https://colab.research.google.com/github/conker84/from-0-to-graph-hero/blob/main/from_0_to_graph_hero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# From zero to a graph hero

In this talk about why graphs are important, and how we can create a simple recommendation engine starting from few CSV files.

# Prerequisites 

Please install all the required dependencies before executing the notebook

In [2]:
from google.colab import output
output.enable_custom_widget_manager()

In [3]:
!pip install neo4j
!pip install ipycytoscape
!pip install networkx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 5.1 MB/s 
  Created wheel for neo4j: filename=neo4j-4.4.4-py3-none-any.whl size=116554 sha256=f0d7c039878e549ab65fba6bc276b59f468e2b3fd4be1c6b19e568a93d362172
  Stored in directory: /root/.cache/pip/wheels/cf/c9/60/dab99fdca0093b46a9c9f5d5b99317c9d323c97d2f5af24e23
Successfully built neo4j
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.6 MB 8.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from getpass import getpass
from neo4j import GraphDatabase

# neo4j_user = input('Neo4j user: ')
# neo4j_password = getpass('Neo4j password: ')
# neo4j_uri = input('Neo4j uri: ')

neo4j_uri = "neo4j+s://a7e3db26.databases.neo4j.io:7687" # put your neo4j url here
neo4j_user = "neo4j" # put your neo4j user here
neo4j_password = "p18n4lernvEXfA2T4Hef3eqy64xFHrvKxFCOrCgRHHc" # put your neo4j password here

neo4j_driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

In [17]:
import pandas as pd
import networkx as nx
from IPython.core.magic import (register_line_magic, register_cell_magic)
import ipycytoscape
from neo4j.graph import Graph

colors = {
  ':Customer': '#fffb00',
  ':Order': '#00f900',
  ':Product': '#ff2600',
  ':Category': '#53d5fd'
}
captions =  {
  ':Customer': 'companyName',
  ':Order': 'shipName',
  ':Product': 'productName',
  ':Category': 'categoryName'
}
node_centered = {
  'selector': 'node',
  'style': {
    'font-size': '10',
    'label': 'data(title)',
    'height': '60',
    'width': '60',
    'text-max-width': '60',
    'text-wrap': 'wrap',
    'text-valign': 'center',
    'background-color': 'data(color)',
    'background-opacity': 0.6,
    'border-width': 3,
    'border-color': '#D3D3D3'
  }
}
edge_directed_named = {
  'selector': 'edge',
  'style': {
    'font-size': '8',
    'label': 'data(label)',
    'line-color': '#D3D3D3',
    'text-rotation': 'autorotate',
    'target-arrow-shape': 'triangle',
    'target-arrow-color': '#D3D3D3',
    'curve-style': 'bezier',
    'text-background-color': "#FCFCFC",
    'text-background-opacity': 0.8,
    'text-background-shape': 'rectangle',
    'width': 'data(weight)'
  }
}
my_style = [node_centered, edge_directed_named]


def to_nextworkx(graph):
  networkx_graph = nx.MultiDiGraph()

  def add_node(node):
    label = ':' + ':'.join(node.labels)
    props = dict(node.items())
    color = colors[label]
    networkx_graph.add_node(node.id, label=label, color=color, properties=props, title=label, tooltip=str(props))


  def add_edge(edge):
    edge_type = edge.type
    props = dict(edge.items())
    networkx_graph.add_edge(edge.start_node.id, edge.end_node.id, weight=2, label=edge.type, tooltip=str(props))
      
  for node in graph._nodes.values():
    add_node(node)

  for rel in graph._relationships.values():
    add_edge(rel)

  return networkx_graph


def display_graph(networkx_graph, config={'layout': 'dagre', 'padding': 0, 'nodeSpacing': 10, 'edgeLengthVal': 10, 'animate': True, 'randomize': True}):
    w = ipycytoscape.CytoscapeWidget()
    w.graph.add_graph_from_networkx(networkx_graph)
    w.set_style(my_style)
    w.set_layout(name=config['layout'],
                 padding=config['padding'],
                 nodeSpacing=config['nodeSpacing'],
                 edgeLengthVal=config['edgeLengthVal'],
                 animate=config['animate'],
                 randomize=config['randomize'],
                 maxSimulations=1500)
    w.set_tooltip_source('tooltip')
    display(w)


def run_query(query):
  # we return only the last one
  with neo4j_driver.session() as session:
    result = None
    for sub_query in query.split(';'):
      sub_query = sub_query.strip()
      if sub_query != "":
        result = session.run(sub_query)
    graph = result.graph()
    if len(graph._nodes) > 0:
      return display_graph(to_nextworkx(graph))
    else:
      return result.to_df()


@register_cell_magic
def cypher(line, cell):
  return run_query(cell)

# Why are graphs important?

We are surrounded by systems that are hopelessly complicated. Consider for example the society that requires cooperation between billions of individuals, or communications infrastructures that integrate billions of cell phones with computers and satellites.

> "I think the next century will be the century of complexity." - Stephen Hawking


These systems are collectively called complex systems, capturing the fact that it is difficult to derive their collective behavior from a knowledge of the system’s components.

Indeed, behind each complex system there is an intricate network that encodes the interactions between the system’s components:

* The network encoding the interactions between genes, proteins, and metabolites integrates these components into live cells. The very existence of this cellular network is a prerequisite of life.
* The wiring diagram capturing the connections between neurons, called the neural network, holds the key to our understanding of how the brain functions and to our consciousness.
* The sum of all professional, friendship, and family ties, often called the social network, is the fabric of the society and determines the spread of knowledge, behavior and resources.
* Communication networks, describing which communication devices interact with each other, through wired internet connections or wireless links, are at the heart of the modern communication system.
* The power grid, a network of generators and transmission lines, supplies with energy virtually all modern technology.
* Trade networks maintain our ability to exchange goods and services, being responsible for the material prosperity that the world has enjoyed since WWII

<img src="https://miro.medium.com/max/1400/1*K6avHhlmtIE0dnGj7whLag.jpeg" alt="Rail Network in Europe by naturalearthdata.com">

*Rail Network in Europe by naturalearthdata.com*

If we want to understand a complex system, we first need to know how its components interact with each other. In other words we need a map of its wiring diagram, and that's when graph (network) came in.

The network representation offers a common language to study systems that may differ greatly in nature, appearance, or scope.

A graph $G$ is a structure $G=<V,E>$ where:
* $V$ are the vertexes or nodes; to each node can be assigned a label (type) and a set of properties as key/value bindings;
* $E$ are the edges; each `edge` has a `source` and `target` node and have a type and a set of properties as key/value bindings.

<img src="https://raw.githubusercontent.com/conker84/from-0-to-graph-hero/main/images/person-follows-person.png" >

# Why do we need a Graph Database?

Relational database are well suited for a lot of use case, but not where your need is to traverse your data, but why?

## JOINs are expensive

Without diving a lot into the problem is known that when you put in join two or more tables, the cost of each join is rough $O(log(n))$ and this means that the performances are proportionally getting worse when the table count gets higher.

Graph databases instead are designed leveraging a structure call [Adiacency Matrix](https://en.wikipedia.org/wiki/Adjacency_matrix) that are specifically designed to reduce the cost of traverse a relationship (given a start node) to $O(1)$

<table>
  <thead>
    <th>Graph</th>
    <th>Matrix representation</th>
  </thead>
  <tbody>
    <td>
      <img src="https://github.com/conker84/from-0-to-graph-hero/blob/main/images/adiacency-graph.png?raw=true" >
    </td>
    <td>
      <img src="https://github.com/conker84/from-0-to-graph-hero/blob/main/images/adiacency-matrix.png?raw=true" >
    </td>
  </tbody>
</table>

## How to query a Graph?

Given the definition of graph above how can you query it?

[openCypher](https://opencypher.org/) (Cypher) is a language specification built to query graphs. Think about it as SQL but for Graphs!




### How does it work?

Cypher has a very nice way in order to represent graphs, it leverages ASCII ART in order to do that, but how?

#### Cypher and ASCII ART

In Cypher:

* `nodes` are represented as `()` and they can also contain identifiers `(person)`
* `relationships` are represented as `-[]-`, they can have an identifier `-[FOLLOWS]-`, a direction `-[]->` or `<-[]-` and the must have a source and a target node `()-[]->()`

Think about at this simple graph:

<img src="https://github.com/conker84/from-0-to-graph-hero/blob/main/images/person-drives-car.png?raw=true" >

The Cypher representation of this is:

`(p:Person)-[d:DRIVES]->(c:Car)`

where:
* `p` is the identifier of the source node which is of type `Person`
* `c` is the identifier of the target node which is of type `Car`
* `d` is the identifier of the relationship which type is `DRIVES`

Given that than you can:

* `CREATE` a graph entity
* `MATCH` a graph entity
* `MERGE` a graph entity; this one is a idempotent operation that first checks if the entity exists otherwise it creates it and return.

```cypher
MATCH path = (p:Person)-[d:DRIVES]->(c:Car)
WHERE p.name = 'Andrea' AND p.surname = 'Santurbano'
RETURN path
```



# Dataset

The Northwind database is an famous dataset containing purchase history that has been used to teach relational databases for years and was a great place to start.

<img src="https://github.com/conker84/from-0-to-graph-hero/blob/main/images/northwind.gif?raw=true" >

It provides us with a rich dataset, but in this what we want to do is to use a subset of information in order to create a subgraph like this:

<img src="https://github.com/conker84/from-0-to-graph-hero/blob/main/images/graph.png?raw=true" />

In [ ]:
%%cypher
// Let's create the constaints
CREATE CONSTRAINT product_id IF NOT EXISTS FOR (p:Product) REQUIRE (p.id) IS UNIQUE;
CREATE CONSTRAINT order_id IF NOT EXISTS FOR (o:Order) REQUIRE (o.id) IS UNIQUE;
CREATE CONSTRAINT customer_id IF NOT EXISTS FOR (c:Customer) REQUIRE (c.id) IS UNIQUE;
CREATE CONSTRAINT category_id IF NOT EXISTS FOR (c:Category) REQUIRE (c.id) IS UNIQUE;

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions


""


In [ ]:
%%cypher
// create the Customer nodes
USING PERIODIC COMMIT 100
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/conker84/from-0-to-graph-hero/main/data/customers.csv" AS row
CREATE (:Customer {id: row.customerID, companyName: row.companyName, fax: row.fax, phone: row.phone});

// create the Order nodes
USING PERIODIC COMMIT 100
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/conker84/from-0-to-graph-hero/main/data/orders.csv" AS row
MERGE (o:Order {id: row.orderID}) ON CREATE SET o.shipName =  row.shipName;

// create the Product nodes
USING PERIODIC COMMIT 100
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/conker84/from-0-to-graph-hero/main/data/products.csv" AS row
CREATE (:Product {productName: row.productName, id: row.productID, unitPrice: toFloat(row.UnitPrice)});

// create the Category nodes
USING PERIODIC COMMIT 100
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/conker84/from-0-to-graph-hero/main/data/categories.csv" AS row
MERGE (c:Category {id: row.categoryID}) ON CREATE SET c.categoryName = row.categoryName, c.description = row.description;

// create the PURCHASED relationships
USING PERIODIC COMMIT 100
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/conker84/from-0-to-graph-hero/main/data/orders.csv" AS row
MATCH (o:Order {id: row.orderID})
MATCH (customer:Customer {id: row.customerID})
MERGE (customer)-[:PURCHASED]->(o);

// create the CONTAINS relationships
USING PERIODIC COMMIT 100
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/conker84/from-0-to-graph-hero/main/data/order-details.csv" AS row
MATCH (o:Order {id: row.orderID})
MATCH (product:Product {id: row.productID})
MERGE (o)-[pu:PRODUCT]->(product)
ON CREATE SET pu.unitPrice = toFloat(row.unitPrice), pu.quantity = toFloat(row.quantity);

// create the HAS relationships
USING PERIODIC COMMIT 100
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/conker84/from-0-to-graph-hero/main/data/products.csv" AS row
MATCH (product:Product {id: row.productID})
MATCH (category:Category {id: row.categoryID})
MERGE (product)-[:HAS]->(category);

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions


""


## Check the graph model

In [26]:
%%cypher
CALL apoc.meta.graph()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'padding': 0, 'nodeSpacing': 10, 'edgeLengthVal': 10, 'anim…

## Visualize a simple graph

In [35]:
%%cypher
match (c:Customer)-[r*..2]->(a)
WHERE c.id = 'CHOPS' AND type(r) NOT IN ['SIMILAR', 'RATED'] 
return *

KeyboardInterrupt: ignored

# Let's build a recommendation engine

Recommender Systems are a type of information filtering system that seek to generate meaningful recommendations to users for items they may be interested in.

## Popular Products

To find the most popular products in the dataset, we can follow the path from `:Customer` to `:Product`


In [34]:
%%cypher
// get all the customers that purchased a product
match (c:Customer)-[:PURCHASED]->(o:Order)-[:PRODUCT]->(p:Product)
// return the company, the product and the number of times that ht bought it
return c.companyName, p.productName, count(o) as orders
order by orders desc
limit 5

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:105: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions


,c.companyName,p.productName,orders
0,Save-a-lot Markets,Chang,5
1,Ernst Handel,Gorgonzola Telino,4
2,Ernst Handel,Guaraná Fantástica,4
3,Ernst Handel,Wimmers gute Semmelknödel,4
4,Ernst Handel,Alice Mutton,4


Nothing so fancy right? Let's do something more graph oriented.

## Content Based Recommendations

The simplest recommendation we can make for a `:Customer` is a content based recommendation.
Based on their previous purchases, can we recommend them anything that they haven't already bought?
For every product our customer has purchased, let's see what other customers have also purchased.
Each `:Product` is related to a `:Category`  so we can use this to further narrow down the list of products to recommend.

**Does it sounds familiar?**

<img src="https://github.com/conker84/from-0-to-graph-hero/blob/main/images/amazon-recommendations.png?raw=true" >


It's quite the same behind what Amazon shows you when you bought a problem and shows corralate products in the same category

In [ ]:
%%cypher
match (c:Customer)-[:PURCHASED]->(o:Order)-[:PRODUCT]->(p:Product)<-[:PRODUCT]-(o2:Order)-[:PRODUCT]->(p2:Product)-[:HAS]->(:Category)<-[:HAS]-(p)
WHERE c.id = 'CHOPS' AND NOT( (c)-[:PURCHASED]->(:Order)-[:PRODUCT]->(p2) )
return c.companyName, p.productName as has_purchased, p2.productName as has_also_purchased, count(DISTINCT o2) as occurrences
order by occurrences desc
limit 5

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:102: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions


,c.companyName,has_purchased,has_also_purchased,occurrences
0,Chop-suey Chinese,Camembert Pierrot,Flotemysost,6
1,Chop-suey Chinese,Tarte au sucre,Pavlova,6
2,Chop-suey Chinese,Camembert Pierrot,Gorgonzola Telino,4
3,Chop-suey Chinese,Guaraná Fantástica,Rhönbräu Klosterbier,3
4,Chop-suey Chinese,Chang,Outback Lager,3


## Collaborative Filtering

<img src="https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif" >

Collaborative filtering (CF) is a technique used by recommender systems and it has two senses:

* a narrow one: collaborative filtering is a method of making automatic predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating);
* a general one: collaborative filtering is the process of filtering for information or patterns using techniques involving collaboration among multiple agents, viewpoints, data sources, etc.

To CF, we can use the k-NN (k-nearest neighbors) Algorithm. k-N works by grouping items into classifications based on their similarity to eachother. In our case, this could be ratings between two Customers for a Product.

The first thing we need to do to make this model work is create some "ratings relationships". For now, let’s create a score somewhere between 0 and 1 for each product based on the number of times a customer has purchased a product.

In [ ]:
%%cypher
// create the RATED relationships
MATCH (c:Customer)-[:PURCHASED]->(o:Order)-[:PRODUCT]->(p:Product)
WITH c, count(p) as total
MATCH (c)-[:PURCHASED]->(o:Order)-[:PRODUCT]->(p:Product)
WITH c, total, p, toFloat(count(o)) as orders
MERGE (c)-[rated:RATED]->(p) SET rated.rating = orders/total
WITH c.companyName as company, p.productName as product, orders, total, rated.rating as rating
ORDER BY rating DESC
RETURN company, product, orders, total, rating
LIMIT 10

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:102: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions


,company,product,orders,total,rating
0,Centro comercial Moctezuma,Sir Rodney's Scones,1.0,2,0.500000
1,Centro comercial Moctezuma,Gravad lax,1.0,2,0.500000
2,Lazy K Kountry Store,Queso Cabrales,1.0,2,0.500000
3,Lazy K Kountry Store,Boston Crab Meat,1.0,2,0.500000
4,North/South,Outback Lager,2.0,6,0.333333
5,GROSELLA-Restaurante,Rhönbräu Klosterbier,1.0,4,0.250000
6,GROSELLA-Restaurante,Ikura,1.0,4,0.250000
7,GROSELLA-Restaurante,Mozzarella di Giovanni,1.0,4,0.250000
8,GROSELLA-Restaurante,Thüringer Rostbratwurst,1.0,4,0.250000
9,Trail's Head Gourmet Provisioners,Tarte au sucre,2.0,9,0.222222


In [ ]:
%%cypher
CALL apoc.meta.graph()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'padding': 0, 'nodeSpacing': 10, 'edgeLengthVal': 10, 'anim…

In [ ]:
%%cypher
MATCH (me:Customer)-[r:RATED]->(p:Product)
WHERE me.id = 'CHOPS'
RETURN p.productName, r.rating
limit 10

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:102: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions


,p.productName,r.rating
0,Vegie-spread,0.090909
1,Gravad lax,0.045455
2,Tarte au sucre,0.090909
3,Singaporean Hokkien Fried Mee,0.045455
4,Ikura,0.045455
5,Sir Rodney's Scones,0.045455
6,Wimmers gute Semmelknödel,0.045455
7,Guaraná Fantástica,0.045455
8,Konbu,0.045455
9,Gnocchi di nonna Alice,0.136364


Now we can use these ratings to compare the preferences of two Customers.


In [28]:
%%cypher
// See Customer's Similar Ratings to Others
MATCH (c1:Customer {id:'CHOPS'})-[r1:RATED]->(p:Product)<-[r2:RATED]-(c2:Customer)
RETURN c1.id, c2.id, p.productName, r1.rating, r2.rating,
CASE WHEN r1.rating-r2.rating < 0 THEN -(r1.rating-r2.rating) ELSE r1.rating-r2.rating END as difference
ORDER BY difference ASC
LIMIT 15

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:105: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions


,c1.id,c2.id,p.productName,r1.rating,r2.rating,difference
0,CHOPS,GREAL,Wimmers gute Semmelknödel,0.045455,0.045455,0.000000
1,CHOPS,BONAP,Wimmers gute Semmelknödel,0.045455,0.045455,0.000000
2,CHOPS,BSBEV,Wimmers gute Semmelknödel,0.045455,0.045455,0.000000
3,CHOPS,MORGK,Vegie-spread,0.090909,0.090909,0.000000
4,CHOPS,BONAP,Manjimup Dried Apples,0.045455,0.045455,0.000000
5,CHOPS,GREAL,Camembert Pierrot,0.045455,0.045455,0.000000
6,CHOPS,GREAL,Chai,0.045455,0.045455,0.000000
7,CHOPS,REGGC,Guaraná Fantástica,0.045455,0.045455,0.000000
8,CHOPS,BSBEV,Manjimup Dried Apples,0.045455,0.045455,0.000000
9,CHOPS,BONAP,Ikura,0.045455,0.045455,0.000000


Now, we can create a similarity score between two Customers using Cosine Similarity

In [29]:
%%cypher
MATCH (c1:Customer)-[r1:RATED]->(p:Product)<-[r2:RATED]-(c2:Customer)
WITH
	SUM(r1.rating*r2.rating) as dot_product,
	SQRT( REDUCE(x=0.0, a IN COLLECT(r1.rating) | x + a^2) ) as r1_length,
	SQRT( REDUCE(y=0.0, b IN COLLECT(r2.rating) | y + b^2) ) as r2_length,
	c1,c2
MERGE (c1)-[s:SIMILARITY]-(c2)
SET s.similarity = dot_product / (r1_length * r2_length)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:105: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions


""


In [30]:
%%cypher
CALL apoc.meta.graph()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'padding': 0, 'nodeSpacing': 10, 'edgeLengthVal': 10, 'anim…

In [31]:
%%cypher
MATCH (me:Customer)-[r:SIMILARITY]->(them)
WHERE me.id='CHOPS'
RETURN me.companyName, them.companyName, r.similarity
ORDER BY r.similarity DESC
limit 10

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:105: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions


,me.companyName,them.companyName,r.similarity
0,Chop-suey Chinese,GROSELLA-Restaurante,1.0
1,Chop-suey Chinese,Eastern Connection,1.0
2,Chop-suey Chinese,Romero y tomillo,1.0
3,Chop-suey Chinese,North/South,1.0
4,Chop-suey Chinese,Ottilies Käseladen,1.0
5,Chop-suey Chinese,Drachenblut Delikatessen,1.0
6,Chop-suey Chinese,Pericles Comidas clásicas,1.0
7,Chop-suey Chinese,Laughing Bacchus Wine Cellars,1.0
8,Chop-suey Chinese,Centro comercial Moctezuma,1.0
9,Chop-suey Chinese,Consolidated Holdings,1.0


Great, let&#8217;s now make a recommendation based on these similarity scores.


In [33]:
%%cypher
WITH 1 as neighbours
MATCH (me:Customer)-[:SIMILARITY]->(c:Customer)-[r:RATED]->(p:Product)
WHERE me.id = 'CHOPS' and NOT ( (me)-[:RATED|PRODUCT|ORDER*1..2]->(p:Product) )
WITH p, COLLECT(r.rating)[0..neighbours] as ratings, collect(c.companyName)[0..neighbours] as customers
WITH p, customers, REDUCE(s=0,i in ratings | s+i) / SIZE(ratings) as recommendation
ORDER BY recommendation DESC
RETURN p.productName, recommendation
LIMIT 10

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:105: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions


,p.productName,customers,recommendation
0,Mishi Kobe Niku,[Consolidated Holdings],0.142857
1,Sirop d'érable,[Wolski Zajazd],0.125000
2,Nord-Ost Matjeshering,[Gourmet Lanchonetes],0.105263
3,Louisiana Fiery Hot Pepper Sauce,[Furia Bacalhau e Frutos do Mar],0.100000
4,Gula Malacca,[Furia Bacalhau e Frutos do Mar],0.100000
5,Flotemysost,[Eastern Connection],0.095238
6,Scottish Longbreads,[Romero y tomillo],0.071429
7,Steeleye Stout,[Romero y tomillo],0.071429
8,Ravioli Angelo,[Romero y tomillo],0.071429
9,Teatime Chocolate Biscuits,[Romero y tomillo],0.071429


There you have it!  Quick and simple recommendations using graph theory and Cypher.
